In [1]:
# !pip3 install opencv-python wandb pandas

In [2]:
import wandb
api = wandb.Api()
entity = "<wandb_entity_name>"
project = "<wandb_project_name>"
wandb_run_name = "<wandb_run_name>"
run_id = wandb_run_name.split('_')[-1]
run = api.run(f"{entity}/{project}/{run_id}")
metrics_dataframe = run.history()

In [3]:
obs_images_path = metrics_dataframe.tail(1)["obs_images"].values[0]['filenames']

In [4]:
import os
import shutil
save_dir = f"{wandb_run_name}/obs_images"
os.makedirs(save_dir, exist_ok=True)
for idx, obs_image_path in enumerate(obs_images_path):
    file = run.file(obs_image_path)
    count = str(idx).zfill(6)
    file.download(replace=True)
    shutil.move(obs_image_path, f"{save_dir}/{count}.png")

In [5]:
import cv2
import glob

# https://docs.opencv.org/4.6.0/dd/d9e/classcv_1_1VideoWriter.html#ac3478f6257454209fa99249cc03a5c59
# https://softron.zendesk.com/hc/en-us/articles/207695697-List-of-FourCC-codes-for-video-codecs
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

output_file_name = f"{wandb_run_name}/run.mp4"
fps = 10
image_size = (128, 128)
video = cv2.VideoWriter(output_file_name,fourcc, fps, image_size)

image_paths = glob.glob(f"{save_dir}/*.png")
image_paths = sorted(image_paths)

for image_path in image_paths:
    img = cv2.imread(image_path)
    video.write(img)
video.release()